## Importing Packages

In [ ]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.Row
import spark.implicits._
import scala.collection.mutable._
import org.apache.spark.graphx.lib._

## Loading DataSet

In [ ]:
// File location and type
val file_location = "violationtraffic.csv"
val file_type = "csv"

// CSV options
val infer_schema = "True"
val first_row_is_header = "True"
val delimiter = ","

// The applied options are for CSV files. For other file types, these will be ignored.
val df = spark.read.format(file_type).option("inferSchema", infer_schema).option("header", first_row_is_header).option("sep", delimiter).load(file_location)


df.limit(20).show()

## Euclidean-LSH

<b> Hash Function (Random Hyperplanes) </b>

In [ ]:
import org.apache.spark.mllib.linalg.SparseVector
import scala.collection.mutable.ArrayBuffer
import scala.util.Random

/**
 * Simple hashing function implements random hyperplane based hash functions described in
 * http://www.cs.princeton.edu/courses/archive/spring04/cos598B/bib/CharikarEstim.pdf
 * r is a random vector. Hash function h_r(u) operates as follows:
 * if r.u < 0 //dot product of two vectors
 *    h_r(u) = 0
 *  else
 *    h_r(u) = 1
 */
class Hasher(val r: Array[Double]) extends Serializable {

  /** hash SparseVector v with random vector r */
  def hash(u : SparseVector) : Int = {
    val rVec: Array[Double] = u.indices.map(i => r(i))
    val hashVal = (rVec zip u.values).map(_tuple => _tuple._1 * _tuple._2).sum
    if (hashVal > 0) 1 else 0
  }

}

object Hasher {

  /** create a new instance providing size of the random vector Array [Double] */
  def apply (size: Int, seed: Long = System.nanoTime) = new Hasher(r(size, seed))

  /** create a random vector whose whose components are -1 and +1 */
  def r(size: Int, seed: Long) : Array[Double] = {
    val buf = new ArrayBuffer[Double]
    val rnd = new Random(seed)
    for (_ <- 0 until size)
      buf += (if (rnd.nextGaussian() < 0) -1 else 1)
    buf.toArray
  }

}

<b> Function to save and Load Lsh Model </b>

In [ ]:
import org.apache.hadoop.fs.Path
import org.apache.spark.SparkContext
import org.apache.spark.mllib.linalg.SparseVector
import org.apache.spark.rdd.RDD
import scala.collection.mutable.ListBuffer
import org.apache.spark.mllib.util.Saveable

import org.json4s._
import org.json4s.JsonDSL._
import org.json4s.jackson.JsonMethods._

/** Helper functions for save/load data from mllib package.
  * TODO: Remove and use Loader functions from mllib. */
object Loader {

  /** Returns URI for path/data using the Hadoop filesystem */
  def dataPath(path: String): String = new Path(path, "data").toUri.toString

  /** Returns URI for path/metadata using the Hadoop filesystem */
  def metadataPath(path: String): String = new Path(path, "metadata").toUri.toString

  /** Returns URI for path/metadata using the Hadoop filesystem */
  def hasherPath(path: String): String = new Path(path, "hasher").toUri.toString

  /**
   * Load metadata from the given path.
   * @return (class name, version, metadata)
   */
  def loadMetadata(sc: SparkContext, path: String): (String, String, JValue) = {
    implicit val formats: DefaultFormats.type = DefaultFormats
    val metadata = parse(sc.textFile(metadataPath(path)).first())
    val clazz = (metadata \ "class").extract[String]
    val version = (metadata \ "version").extract[String]
    (clazz, version, metadata)
  }

}


<b> Main LSHModel Class And Object </b>

In [ ]:
import org.json4s._
import org.json4s.JsonDSL._
import org.json4s.jackson.JsonMethods._



/** Create LSH model for maximum m number of elements in each vector.
  *
  * @param m max number of possible elements in a vector
  * @param numHashFunc number of hash functions
  * @param numHashTables number of hashTables.
  *
  * */
class LSHModel(val m: Int, val numHashFunc : Int, val numHashTables: Int)
  extends Serializable with Saveable {

  /** generate numHashFunc * numBands randomly generated hash functions and store them in hashFunctions */
  private val _hashFunctions = ListBuffer[Hasher]()
  for (_ <- 0 until numHashFunc * numHashTables)
    _hashFunctions += Hasher(m)
  final var hashFunctions: List[(Hasher, Int)] = _hashFunctions.toList.zipWithIndex

  /** the "hashTables" ((hashTableID, hash key), vector_id) */
  var hashTables: RDD[((Int, String), Long)] = _

  /** generic filter function for hashTables. */
  def filter(f: (((Int, String), Long)) => Boolean): RDD[((Int, String), Long)] =
    hashTables.map(a => a).filter(f)

  /** hash a single vector against an existing model and return the candidate buckets */
  def filter(data: SparseVector, model: LSHModel, itemID: Long): RDD[Long] = {
    val hashKey = hashFunctions.map(h => h._1.hash(data)).mkString("")
    hashTables.filter(x => x._1._2 == hashKey).map(a => a._2)
  }

  /** creates hashValue for each hashTable.*/
  def hashValue(data: SparseVector): List[(Int, String)] =
    hashFunctions.map(a => (a._2 % numHashTables, a._1.hash(data)))
    .groupBy(_._1)
    .map(x => (x._1, x._2.map(_._2).mkString(""))).toList

  /** returns candidate set for given vector id.*/
  def getCandidates(vId: Long): RDD[Long] = {
    val buckets = hashTables.filter(x => x._2 == vId).map(x => x._1).distinct().collect()
    hashTables.filter(x => buckets contains x._1).map(x => x._2).filter(x => x != vId)
  }

  /** returns candidate set for given vector.*/
  def getCandidates(v: SparseVector): RDD[Long] = {
    val hashVal = hashValue(v)
    hashTables.filter(x => hashVal contains x._1).map(x => x._2)
  }

  /** adds a new sparse vector with vector Id: vId to the model. */
  def add (vId: Long, v: SparseVector, sc: SparkContext): LSHModel = {
    val newRDD = sc.parallelize(hashValue(v).map(a => (a, vId)))
    hashTables ++ newRDD
    this
  }

  /** remove sparse vector with vector Id: vId from the model. */
  def remove (vId: Long, sc: SparkContext): LSHModel = {
    hashTables =  hashTables.filter(x => x._2 != vId)
    this
  }

      
  override def save(sc: SparkContext, path: String): Unit = LSHModel.SaveLoadV0_0_1.save(sc, this, path)

  def formatVersion: String = "0.0.1"

}

object LSHModel {

  def load(sc: SparkContext, path: String): LSHModel = {
    LSHModel.SaveLoadV0_0_1.load(sc, path)
  }

  private object SaveLoadV0_0_1 {

    private val thisFormatVersion = "0.0.1"
    private val thisClassName = this.getClass.getName

    def save(sc: SparkContext, model: LSHModel, path: String): Unit = {

      val metadata = compact(render(("class" -> thisClassName) ~ ("version" -> thisFormatVersion)))

      //save metadata info
      sc.parallelize(Seq(metadata), 1).saveAsTextFile(Loader.metadataPath(path))

      //save hash functions as (hashTableId, randomVector)
      sc.parallelize(model.hashFunctions.map(x => (x._2, x._1.r.mkString(","))).map(_.productIterator.mkString(","))).saveAsTextFile(Loader.hasherPath(path))

     //save data as (hashTableId#, hashValue, vectorId)
      model.hashTables.map(x => (x._1._1, x._1._2, x._2)).map(_.productIterator.mkString(",")).saveAsTextFile(Loader.dataPath(path))

    }

    def load(sc: SparkContext, path: String): LSHModel = {

      implicit val formats: DefaultFormats.type = DefaultFormats
      val (className, formatVersion, _) = Loader.loadMetadata(sc, path)
      assert(className == thisClassName)
      assert(formatVersion == thisFormatVersion)
      val hashTables = sc.textFile(Loader.dataPath(path)).map(x => x.split(",")).map(x => ((x(0).toInt, x(1)), x(2).toLong))
      val hashers = sc.textFile(Loader.hasherPath(path)).map(a => a.split(",")).map(x => (x.head, x.tail)).map(x => (new Hasher(x._2.map(_.toDouble)), x._1.toInt)).collect().toList
      val numBands = hashTables.map(x => x._1._1).distinct.count()
      val numHashFunc = hashers.size / numBands

      //Validate loaded data
      //check size of data
      assert(hashTables.count != 0, s"Loaded hashTable data is empty")
      //check size of hash functions
      assert(hashers.nonEmpty, s"Loaded hasher data is empty")
      //check hashValue size. Should be equal to numHashFunc
      assert(hashTables.map(x => x._1._2).filter(x => x.length != numHashFunc).collect().length == 0,s"hashValues in data does not match with hash functions")

      //create model
      val model = new LSHModel(0, numHashFunc.toInt, numBands.toInt)
      model.hashFunctions = hashers
      model.hashTables = hashTables

      model
    }
  }
}


class LSH(data : RDD[(Long, SparseVector)] = null, m: Int = 0, numHashFunc : Int = 4, numHashTables : Int = 4) extends Serializable {

  def run() : LSHModel = {

    //create a new model object
    val model = new LSHModel(m, numHashFunc, numHashTables)

    val dataRDD = data.cache()

    //compute hash keys for each vector
    // - hash each vector numHashFunc times
    // - concat each hash value to create a hash key
    // - position hashTable id hash keys and vector id into a new RDD.
    // - creates RDD of ((hashTable#, hash_key), vec_id) tuples.
    model.hashTables = dataRDD.map(v => (model.hashFunctions.map(h => (h._1.hash(v._2), h._2 % numHashTables)), v._1)).map(x => x._1.map(a => ((a._2, x._2), a._1))).flatMap(a => a).groupByKey().map(x => ((x._1._1, x._2.mkString("")), x._1._2)).cache()

    model

  }

  def cosine(a: SparseVector, b: SparseVector): Double = {
    val intersection = a.indices.intersect(b.indices)
    val magnitudeA = intersection.map(x => Math.pow(a.apply(x), 2)).sum
    val magnitudeB = intersection.map(x => Math.pow(b.apply(x), 2)).sum
    intersection.map(x => a.apply(x) * b.apply(x)).sum / (Math.sqrt(magnitudeA) * Math.sqrt(magnitudeB))
  }

}




<b> Creating (time, car, cameraId) Df </b>

In [ ]:
var basket_holder = df.select(col("PassDatetime").alias("time"), col("MasterPlateNumber").alias("car"), col("DeviceId").alias("camera"))
.withColumn("time", dayofyear(col("time")))

basket_holder.show(3)

<b> Create Dataframe of distinct Camera Ids </b>

In [ ]:
//list of distinct items
val items_count: Int = df.select(col("DeviceId")).distinct().count().toInt
val distinct_cameras = df.select(col("DeviceId").alias("camera")).distinct()
println("items_count: " + items_count)

<b> Create Hash Df to map value of cameraId </b>

In [ ]:
val distinct_cameras_hash = distinct_cameras.rdd.zipWithIndex.map(x => (x._1(0).asInstanceOf[Int], x._2.asInstanceOf[Int])).toDF("camera", "id")
distinct_cameras_hash.show(3)

<b>Saving Table Of CameraId Hash </b>

In [ ]:
distinct_cameras_hash.coalesce(1).write.format("csv").option("header", "true").save("camera_id.csv")

<b> Replace CameraId with New Hash </b>

In [ ]:
val new_basket_holder = basket_holder.join(distinct_cameras_hash, "camera").drop("camera")
new_basket_holder.show(3)

<b> First Building path of each car Then Make it sparse </b>

In [ ]:
import org.apache.spark.mllib.linalg.Vectors

//row(1).asInstanceOf[Int].longValue*1000 + row(0).asInstanceOf[Int].longValue
val hour_basket = new_basket_holder.rdd.map(row => (row(1).asInstanceOf[Int],(row(2).asInstanceOf[Int], 1.0)))


val hour_basket_grouped = hour_basket.distinct.groupByKey()
//hour_basket_grouped.take(3)
//convert each car's camera list to tuple of (car_id, SparseVector_of_cameras)
val sparseVectorData = hour_basket_grouped.map(a=>(a._1.asInstanceOf[Long], Vectors.sparse(items_count.asInstanceOf[Int], a._2.toSeq).asInstanceOf[SparseVector]))

sparseVectorData.take(1)

<b> Setup LSH </b>

In [ ]:
val lsh = new LSH(sparseVectorData, items_count.asInstanceOf[Int], numHashFunc = 8, numHashTables = 15)
val model = lsh.run()

<b> Show Nearest Neighbors Candidate </b>

In [ ]:
val candList = model.getCandidates(762817122)
println("Number of Candidates: "+ candList.count())
println("Candidate List: " + candList.collect().toList)

## <b> Graphx And Community Detection </b>

<b> GroupBy Data Base On Car and DayOfYear </b>

In [ ]:
val path_df = df.select(col("PassDatetime").alias("time"), col("MasterPlateNumber").alias("car"), col("DeviceId").alias("camera")).groupBy(col("car"), dayofyear(col("time"))).agg(collect_set(struct("time", "camera")).alias("list_col"))

<b> Sort Paths base On PassDateTime so we have a directed Path </b>

In [ ]:
val path_find = udf ((row_detail: Seq[Row]) =>  {
    

  val arr1Tup: Seq[(String, Int)] = row_detail.map{case Row(s:String,i:Int) => (s,i)}
  val res = arr1Tup.sortBy(_._1)
  res.map(x => x._2)
})


val path_df2 = path_df.withColumn("paths", path_find(col("list_col")))
val path_df3 = path_df2.filter(size($"paths") > 1)

<b> Building Edge Tuples base on Adjacency List </b>

In [ ]:
def path_build(row_detail: WrappedArray[Int]): Array[Tuple2[Long, Long]] = {

  var res : Array[Tuple2[Long, Long]] = Array()
    
  for (i <- 0 until row_detail.length - 1){
      
     res :+= (row_detail(i).asInstanceOf[Long],row_detail(i+1).asInstanceOf[Long])
  }
    res
  
}

val path_df4 = path_df3.select(col("car"), col("paths")).rdd.flatMap(x => path_build(x(1).asInstanceOf[WrappedArray[Int]]))//.reduceByKey(_ | _)

<b> Build Graph base on Edge Tuples </b>

In [ ]:
val graph = Graph.fromEdgeTuples(path_df4, 1)

<b> PageRank Algorithm </b>

In [ ]:
// Run PageRank
val ranks = graph.pageRank(0.0001, 0.15).vertices

<b> Strongly Connected Componrnts Algorithm </b>

In [ ]:
val connected_components = graph.stronglyConnectedComponents(5)

<b> Extract Strongly Components as Array[List] </b>

In [ ]:
val strong_components = connected_components.vertices.map(_.swap).groupByKey.map(_._2).map(x => x.toList)
strong_components.take(1)

<b> Convert Array[List] to DF so we can save it as CSV </b>

In [ ]:
var strong_components_df = spark.createDataFrame(strong_components).withColumn("Community", col("value")).drop("value")

strong_components_df = strong_components_df.withColumn("Community", col("Community").cast("String"))

println("some communities : ")
strong_components_df.show(3)

<b> Save SCG Result as CSV </b>

In [ ]:
strong_components_df.coalesce(1).write.format("csv").option("header", "true").save("Strong_Community_Detection.csv")

<b> LabelPropagation Algorithm </b>

In [ ]:
import scala.collection.{mutable, Map}
import scala.reflect.ClassTag

import org.apache.spark.graphx._

/** Label Propagation algorithm. */
object LabelPropagation {
  /**
   * Run static Label Propagation for detecting communities in networks.
   *
   * Each node in the network is initially assigned to its own community. At every superstep, nodes
   * send their community affiliation to all neighbors and update their state to the mode community
   * affiliation of incoming messages.
   *
   * LPA is a standard community detection algorithm for graphs. It is very inexpensive
   * computationally, although (1) convergence is not guaranteed and (2) one can end up with
   * trivial solutions (all nodes are identified into a single community).
   *
   * @tparam ED the edge attribute type (not used in the computation)
   *
   * @param graph the graph for which to compute the community affiliation
   * @param maxSteps the number of supersteps of LPA to be performed. Because this is a static
   * implementation, the algorithm will run for exactly this many supersteps.
   *
   * @return a graph with vertex attributes containing the label of community affiliation
   */
  def run[VD, ED: ClassTag](graph: Graph[VD, ED], maxSteps: Int): Graph[VertexId, ED] = {
    require(maxSteps > 0, s"Maximum of steps must be greater than 0, but got ${maxSteps}")

    val lpaGraph = graph.mapVertices { case (vid, _) => vid }
    def sendMessage(e: EdgeTriplet[VertexId, ED]): Iterator[(VertexId, Map[VertexId, Long])] = {
      Iterator((e.srcId, Map(e.dstAttr -> 1L)), (e.dstId, Map(e.srcAttr -> 1L)))
    }
    def mergeMessage(count1: Map[VertexId, Long], count2: Map[VertexId, Long])
      : Map[VertexId, Long] = {
      // Mimics the optimization of breakOut, not present in Scala 2.13, while working in 2.12
      val map = mutable.Map[VertexId, Long]()
      (count1.keySet ++ count2.keySet).foreach { i =>
        val count1Val = count1.getOrElse(i, 0L)
        val count2Val = count2.getOrElse(i, 0L)
        map.put(i, count1Val + count2Val)
      }
      map
    }
    def vertexProgram(vid: VertexId, attr: Long, message: Map[VertexId, Long]): VertexId = {
      if (message.isEmpty) attr else message.maxBy(_._2)._1
    }
    val initialMessage = Map[VertexId, Long]()
    Pregel(lpaGraph, initialMessage, maxIterations = maxSteps)(
      vprog = vertexProgram,
      sendMsg = sendMessage,
      mergeMsg = mergeMessage)
  }
}

<b> Run it with 5 iteration </b>

In [ ]:
val lp =  LabelPropagation
val labeld_graph = lp.run(graph, 5)

<b> CameraId with CommunityId Tuples </b>

In [ ]:
labeld_graph.vertices.take(10)

<b> GroupBy CommunityId So we make Df </b>

In [ ]:
val communities = labeld_graph.vertices.map(x => (x._2, Array(x._1))).reduceByKey(_++_)


<b> Make Df of Communities </b>

In [ ]:
var communitiesDf = communities.toDF("CommunityId", "CameraId")

communitiesDf = communitiesDf.withColumn("CameraId", col("CameraId").cast("String"))

println("some communities : ")
communitiesDf.show(3)

<b> Save communities as CSV </b>

In [ ]:
communitiesDf.coalesce(1).write.format("csv").option("header", "true").save("Community_Detection.csv")